In [18]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from selenium import webdriver

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [4]:
Tesla = yf.Ticker('TSLA')
tesla_data = Tesla.history(period = "max")
tesla_data.reset_index(inplace = True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


Question 2

In [8]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

html_data = requests.get(url, headers=headers).text
soup = BeautifulSoup(html_data, "html.parser")

In [10]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

tbody_elements = soup.find_all("tbody")

if len(tbody_elements) >= 2:
    for row in tbody_elements[1].find_all("tr"):
        col = row.find_all("td")
        date = col[0].text
        revenue = col[1].text.replace("$", "").replace(",", "")
        tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date": [date], "Revenue": [revenue]})], ignore_index=True)
else:
    print("The second tbody element does not exist or is empty.")

In [11]:
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]
tesla_revenue.tail()

,Date,Revenue
50,2010-09-30,31
51,2010-06-30,28
52,2010-03-31,21
54,2009-09-30,46
55,2009-06-30,27


Question 3

In [12]:
GameStop = yf.Ticker("GME")
gme_data = GameStop.history(period = 'max')
gme_data.reset_index(inplace = True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578048,1.607505,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


Question 4

In [23]:
driver = webdriver.Chrome()  

url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
driver.get(url)

driver.implicitly_wait(10)

# Get the page source after dynamic content is loaded
html_data = driver.page_source

# Close the browser
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(html_data, "html.parser")

gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
tbody_elements = soup.find_all("tbody")

# Check if tbody_elements contains at least one element
if len(tbody_elements) >= 1:
    for row in tbody_elements[0].find_all("tr"):
        col = row.find_all("td")
        date = col[0].text
        revenue = col[1].text.replace("$", "").replace(",", "")
        gme_revenue.loc[len(gme_revenue)] = [date, revenue]
else:
    print("No tbody element exists or all tbody elements are empty.")

# Display the scraped data
print(gme_revenue)

    Date Revenue
0   2023    5927
1   2022    6011
2   2021    5090
3   2020    6466
4   2019    8285
5   2018    8547
6   2017    7965
7   2016    9364
8   2015    9296
9   2014    9040
10  2013    8887
11  2012    9551
12  2011    9474
13  2010    9078
14  2009    8806


Question 5

In [30]:
def make_graph(stock_data, revenue_data, stock):
    trace_stock = go.Scatter(x=stock_data['Date'], y=stock_data['Close'], mode='lines', name='Stock Price')
    trace_revenue = go.Bar(x=revenue_data['Date'], y=revenue_data['Revenue'], name='Revenue')

    fig = go.Figure(data=[trace_stock, trace_revenue])
    fig.update_layout(
        title_text=f"{stock} Stock Price and Revenue",
        xaxis_title="Date",
        yaxis_title="Price / Revenue",
        height=600,
        xaxis_rangeslider_visible=True
    )

    # Display the figure
    display(fig)

    # Save the figure as an image (optional)
    fig.write_image("plot.png")



In [31]:
make_graph(tesla_data, tesla_revenue, 'Tesla')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'lines',
              'name': 'Stock Price',
              'type': 'scatter',
              'x': array([datetime.datetime(2010, 6, 29, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2010, 6, 30, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2010, 7, 1, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          ...,
                          datetime.datetime(2023, 7, 18, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 7, 19, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 7, 20, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)],
                         dtype=object),
              'y': array([  1.59266698,   1.58866704,   1.46399999, ..., 293.33999634,
                          291.26000977, 262.8999939 ])},
             {'name': 'Revenue',
              'type': 'bar',
              'x': array(['2023-03-31', '2022-12-31', '2022-09-30', '2022-06-30', '2022-03-31',
                          '2021-12-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-12-31',
                          '2020-09-30', '2020-06-30', '2020-03-31', '2019-12-31', '2019-09-30',
                          '2019-06-30', '2019-03-31', '2018-12-31', '2018-09-30', '2018-06-30',
                          '2018-03-31', '2017-12-31', '2017-09-30', '2017-06-30', '2017-03-31',
                          '2016-12-31', '2016-09-30', '2016-06-30', '2016-03-31', '2015-12-31',
                          '2015-09-30', '2015-06-30', '2015-03-31', '2014-12-31', '2014-09-30',
                          '2014-06-30', '2014-03-31', '2013-12-31', '2013-09-30', '2013-06-30',
                          '2013-03-31', '2012-12-31', '2012-09-30', '2012-06-30', '2012-03-31',
                          '2011-12-31', '2011-09-30', '2011-06-30', '2011-03-31', '2010-12-31',
                          '2010-09-30', '2010-06-30', '2010-03-31', '2009-09-30', '2009-06-30'],
                         dtype=object),
              'y': array(['23329', '24318', '21454', '16934', '18756', '17719', '13757', '11958',
                          '10389', '10744', '8771', '6036', '5985', '7384', '6303', '6350',
                          '4541', '7226', '6824', '4002', '3409', '3288', '2985', '2790', '2696',
                          '2285', '2298', '1270', '1147', '1214', '937', '955', '940', '957',
                          '852', '769', '621', '615', '431', '405', '562', '306', '50', '27',
                          '30', '39', '58', '58', '49', '36', '31', '28', '21', '46', '27'],
                         dtype=object)}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'Tesla Stock Price and Revenue'},
               'xaxis': {'rangeslider': {'visible': True}, 'title': {'text': 'Date'}},
               'yaxis': {'title': {'text': 'Price / Revenue'}}}
})

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [32]:
make_graph(gme_data, gme_revenue, 'GameStop')


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'lines',
              'name': 'Stock Price',
              'type': 'scatter',
              'x': array([datetime.datetime(2002, 2, 13, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2002, 2, 14, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          datetime.datetime(2002, 2, 15, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EST-1 day, 19:00:00 STD>),
                          ...,
                          datetime.datetime(2023, 7, 18, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 7, 19, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>),
                          datetime.datetime(2023, 7, 20, 0, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)],
                         dtype=object),
              'y': array([ 1.69166672,  1.68325019,  1.67483425, ..., 23.07999992, 23.17000008,
                          22.90999985])},
             {'name': 'Revenue',
              'type': 'bar',
              'x': array(['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015',
                          '2014', '2013', '2012', '2011', '2010', '2009'], dtype=object),
              'y': array(['5927', '6011', '5090', '6466', '8285', '8547', '7965', '9364', '9296',
                          '9040', '8887', '9551', '9474', '9078', '8806'], dtype=object)}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'GameStop Stock Price and Revenue'},
               'xaxis': {'rangeslider': {'visible': True}, 'title': {'text': 'Date'}},
               'yaxis': {'title': {'text': 'Price / Revenue'}}}
})

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
